In [3]:
%pip install rasterio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
"""
Data Preparation - Carga y limpieza de datos
Ejecuta directamente, genera outputs
"""

import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from pathlib import Path
import unicodedata
import warnings
warnings.filterwarnings('ignore')

### Configurar rutas
BASE_DIR = Path("C:/Users/ASUS/OneDrive - Universidad del Pacífico/Tareas Data Science/Minimum-Temperature-Raster")
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUT_DIR = DATA_DIR / 'outputs'

### Crear directorios si no existen
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("PASO 1: PREPARACION DE DATOS")
print("=" * 70)

### CARGAR SHAPEFILE DE DISTRITOS
print("\n[1/4] Cargando shapefile de distritos...")

shapefile_path = RAW_DIR / 'shape_file' / 'DISTRITOS.shp'
print(f"Intentando cargar shapefile desde: {shapefile_path}")
districts = gpd.read_file(shapefile_path)
districts = gpd.read_file(shapefile_path)
print(f"Distritos cargados: {len(districts)}")
print(f"Columnas originales: {list(districts.columns)[:5]}...")

### Normalizar nombres (sin tildes, mayúsculas)
def clean_text(text):
    if pd.isna(text):
        return text
    text = str(text)
    normalized = unicodedata.normalize('NFD', text)
    no_accents = ''.join(c for c in normalized if unicodedata.category(c) != 'Mn')
    return no_accents.upper().strip()

### Renombrar y limpiar campos
if 'IDDIST' in districts.columns:
    districts['UBIGEO'] = districts['IDDIST'].astype(str).str.replace('.0', '').astype(int)
else:
    districts['UBIGEO'] = range(1, len(districts) + 1)

### Buscar campo de nombre
name_col = next((c for c in ['NOMBDIST', 'DISTRITO', 'NAME'] if c in districts.columns), None)
if name_col:
    districts['NAME'] = districts[name_col].apply(clean_text)
else:
    districts['NAME'] = [f"DISTRITO_{i}" for i in range(len(districts))]

### Buscar campo de región/departamento
region_col = next((c for c in ['DEPARTAMEN', 'DEPARTAMENTO', 'NOMBDEP'] if c in districts.columns), None)
if region_col:
    districts['REGION'] = districts[region_col].apply(clean_text)
else:
    districts['REGION'] = 'PERU'

### Mantener solo columnas esenciales
districts = districts[['NAME', 'REGION', 'UBIGEO', 'geometry']].copy()

### Reparar geometrías inválidas
invalid_count = (~districts.geometry.is_valid).sum()
if invalid_count > 0:
    print(f"Reparando {invalid_count} geometrias invalidas...")
    districts['geometry'] = districts.geometry.buffer(0)

print(f"Regiones unicas: {districts['REGION'].nunique()}")
print(f"\nMuestra de datos:")
print(districts[['NAME', 'REGION', 'UBIGEO']].head())

### Guardar distritos procesados
districts_out = PROCESSED_DIR / 'peru_districts.gpkg'
districts.to_file(districts_out, driver='GPKG')
print(f"\nGuardado: {districts_out.name}")

### CARGAR Y ANALIZAR RASTER
print("\n[2/4] Analizando raster de temperatura...")

raster_path = RAW_DIR / 'tmin_raster.tif'

with rasterio.open(raster_path) as src:
    print(f"Bandas: {src.count}")
    print(f"Dimensiones: {src.shape}")
    print(f"CRS: {src.crs}")
    print(f"Tipo de dato: {src.dtypes[0]}")
    
    ### Leer primera banda para análisis
    band_1 = src.read(1)
    valid = band_1[~np.isnan(band_1)]
    
    data_min = np.min(valid)
    data_max = np.max(valid)
    data_mean = np.mean(valid)
    
    print(f"\nRango de datos: {data_min:.2f} a {data_max:.2f}")
    print(f"Media: {data_mean:.2f}")
    
    ### Determinar si está escalado
    if data_max > 50:
        scale_factor = 0.1 if data_max < 500 else 0.01
        print(f"Datos escalados detectados. Factor: {scale_factor}")
    else:
        scale_factor = 1.0
        print("Datos en grados Celsius reales")
    
    ### Guardar metadata
    metadata = {
        'n_bands': src.count,
        'crs': str(src.crs),
        'scale_factor': scale_factor,
        'data_min': data_min,
        'data_max': data_max,
        'data_mean': data_mean
    }

### VALIDAR COMPATIBILIDAD
print("\n[3/4] Validando compatibilidad CRS...")

with rasterio.open(raster_path) as src:
    raster_crs = src.crs
    
if districts.crs != raster_crs:
    print(f"CRS diferentes: {districts.crs} vs {raster_crs}")
    print("Se reproyectara automaticamente durante analisis")
else:
    print("CRS compatible")

### GUARDAR METADATA
print("\n[4/4] Guardando metadata...")

metadata_df = pd.DataFrame([metadata])
metadata_path = PROCESSED_DIR / 'raster_metadata.csv'
metadata_df.to_csv(metadata_path, index=False)
print(f"Guardado: {metadata_path.name}")

### RESUMEN FINAL
print("\n" + "=" * 70)
print("PREPARACION COMPLETADA")
print("=" * 70)
print(f"Distritos procesados: {len(districts)}")
print(f"Regiones: {districts['REGION'].nunique()}")
print(f"Factor de escala: {scale_factor}")
print(f"\nArchivos generados:")
print(f"  - {districts_out.name}")
print(f"  - {metadata_path.name}")
print("=" * 70)

PASO 1: PREPARACION DE DATOS

[1/4] Cargando shapefile de distritos...
Intentando cargar shapefile desde: C:\Users\ASUS\OneDrive - Universidad del Pacífico\Tareas Data Science\Minimum-Temperature-Raster\data\raw\shape_file\DISTRITOS.shp
Distritos cargados: 1873
Columnas originales: ['IDDPTO', 'DEPARTAMEN', 'IDPROV', 'PROVINCIA', 'IDDIST']...
Regiones unicas: 25

Muestra de datos:
                     NAME   REGION  UBIGEO
0         CODO DEL POZUZO  HUANUCO  100902
1             TOURNAVISTA  HUANUCO  100904
2  ALEXANDER VON HUMBOLDT  UCAYALI  250305
3                 IRAZOLA  UCAYALI  250302
4                 NESHUYA  UCAYALI  250304

Guardado: peru_districts.gpkg

[2/4] Analizando raster de temperatura...
Bandas: 5
Dimensiones: (397, 285)
CRS: EPSG:4326
Tipo de dato: float32

Rango de datos: -9.05 a 24.64
Media: 18.01
Datos en grados Celsius reales

[3/4] Validando compatibilidad CRS...
CRS compatible

[4/4] Guardando metadata...
Guardado: raster_metadata.csv

PREPARACION COMPLETADA
Di